In [38]:
!pip install catboost

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import root_mean_squared_log_error
from collections import defaultdict, deque

In [40]:
pd.options.display.float_format = '{:.4f}'.format
pd.options.display.max_columns = None

# Prepare Data

In [41]:
data = pd.read_csv('drive/MyDrive/data/store_sales_ecuador/data.csv')
data.head()

,date,store_nbr,family,id,sales,onpromotion,year,month,day_of_week,sin_7,cos_7,sin_30.5,cos_30.5,sin_365.25,cos_365.25,is_payday,is_weekend,is_christmas,earthquake_effect,sales_lag_1,sales_lag_7,sales_lag_15,sales_lag_30,sales_lag_365,sales_rolling_mean_30,sales_rolling_std_30,sales_rolling_mean_15,sales_rolling_std_15,sales_rolling_median_7,sales_rolling_mean_7,store_city,store_state,store_type,store_cluster,holiday_locale,is_holiday,is_event,store_in_event_area,dcoilwtico,oil_lag_1,oil_lag_7,oil_lag_15,oil_lag_30,oil_lag_365,oil_rolling_mean_30,oil_rolling_std_30,oil_rolling_mean_15,oil_rolling_std_15,oil_rolling_median_7,oil_rolling_mean_7,transactions
0,2013-01-01,1,AUTOMOTIVE,0.0000,0.0000,0.0000,2013,1,1,0.7818,0.6235,0.2046,0.9789,0.0172,0.9999,0,0,1,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1,93.1400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
1,2013-01-01,1,BABY CARE,1.0000,0.0000,0.0000,2013,1,1,0.7818,0.6235,0.2046,0.9789,0.0172,0.9999,0,0,1,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1,93.1400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
2,2013-01-01,1,BEAUTY,2.0000,0.0000,0.0000,2013,1,1,0.7818,0.6235,0.2046,0.9789,0.0172,0.9999,0,0,1,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1,93.1400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
3,2013-01-01,1,BEVERAGES,3.0000,0.0000,0.0000,2013,1,1,0.7818,0.6235,0.2046,0.9789,0.0172,0.9999,0,0,1,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1,93.1400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
4,2013-01-01,1,BOOKS,4.0000,0.0000,0.0000,2013,1,1,0.7818,0.6235,0.2046,0.9789,0.0172,0.9999,0,0,1,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1,93.1400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000


In [42]:
data.isnull().sum()

,0
date,0
store_nbr,0
family,0
id,7128
sales,0
onpromotion,0
year,0
month,0
day_of_week,0
sin_7,0


I realized that I was leaking data through lag features and rolling statistics. There are two ways to go:

1) Remove lag features and rolling statistics for 1 and 7 days - the model will predict 15 days in advance (as provided in the test sample)  
2) Predict the model for the day ahead, add lag features and rolling statistics as the forecast progresses

I choose option 2.

In [43]:
data['date'] = pd.to_datetime(data['date'])
end_date = data.loc[data.index[-1], 'date']
DAYS_TO_PREDICT = 15
end_of_train = end_date - pd.Timedelta(days=DAYS_TO_PREDICT)
start_valid_date = end_of_train + pd.Timedelta(days=1)
print(end_of_train)

2017-07-31 00:00:00


Fix the data leak (we leave oil as it is - we believe that the data on future oil prices came from another model.)

In [44]:
data.loc[data['date'] >= end_of_train + pd.Timedelta(days=1), ['sales_lag_1']] = np.nan
data.loc[
    data['date'] >= end_of_train + pd.Timedelta(days=7),
     ['sales_lag_7', 'sales_rolling_median_7', 'sales_rolling_mean_7']] = np.nan
data[(data['family'] == 'AUTOMOTIVE') & (data['store_nbr'] == 1)  & (data['date'] > end_of_train)]

,date,store_nbr,family,id,sales,onpromotion,year,month,day_of_week,sin_7,cos_7,sin_30.5,cos_30.5,sin_365.25,cos_365.25,is_payday,is_weekend,is_christmas,earthquake_effect,sales_lag_1,sales_lag_7,sales_lag_15,sales_lag_30,sales_lag_365,sales_rolling_mean_30,sales_rolling_std_30,sales_rolling_mean_15,sales_rolling_std_15,sales_rolling_median_7,sales_rolling_mean_7,store_city,store_state,store_type,store_cluster,holiday_locale,is_holiday,is_event,store_in_event_area,dcoilwtico,oil_lag_1,oil_lag_7,oil_lag_15,oil_lag_30,oil_lag_365,oil_rolling_mean_30,oil_rolling_std_30,oil_rolling_mean_15,oil_rolling_std_15,oil_rolling_median_7,oil_rolling_mean_7,transactions
3032964,2017-08-01,1,AUTOMOTIVE,2974158.0000,5.0000,0.0000,2017,8,1,0.4339,-0.9010,-0.1028,0.9947,-0.4991,-0.8666,0,0,0,0.0000,NaN,10.0000,2.0000,4.0000,3.0000,4.6667,2.8203,5.0000,3.1848,5.0000,5.2857,Quito,Pichincha,D,13,NONE,0,0,0,49.1900,50.2100,47.7700,46.0200,46.0200,40.0500,46.5533,1.7222,47.6380,1.6941,49.7200,49.2529,1795.0000
3034746,2017-08-02,1,AUTOMOTIVE,2975940.0000,4.0000,0.0000,2017,8,2,-0.4339,-0.9010,0.1028,0.9947,-0.5139,-0.8578,0,0,0,0.0000,NaN,2.0000,3.0000,0.0000,10.0000,4.7000,2.8180,5.2000,3.0752,5.0000,4.5714,Quito,Pichincha,D,13,NONE,0,0,0,49.6000,49.1900,48.5800,46.4000,46.0200,39.5000,46.6590,1.7844,47.8493,1.6754,49.7200,49.4557,1892.0000
3036528,2017-08-03,1,AUTOMOTIVE,2977722.0000,3.0000,0.0000,2017,8,3,-0.9749,-0.2225,0.3041,0.9526,-0.5286,-0.8489,0,0,0,0.0000,NaN,5.0000,7.0000,5.0000,0.0000,4.8333,2.6792,5.2667,3.0347,5.0000,4.8571,Quito,Pichincha,D,13,NONE,0,0,0,49.0300,49.6000,49.0500,47.1000,46.0200,40.8000,46.7783,1.8584,48.0627,1.6814,49.7200,49.6014,1726.0000
3038310,2017-08-04,1,AUTOMOTIVE,2979504.0000,8.0000,0.0000,2017,8,4,-0.7818,0.6235,0.4925,0.8703,-0.5431,-0.8397,0,0,0,0.0000,NaN,7.0000,4.0000,1.0000,3.0000,4.7667,2.6997,5.0000,3.0472,4.0000,4.5714,Quito,Pichincha,D,13,NONE,0,0,0,49.5700,49.0300,49.7200,46.7300,45.1100,41.9200,46.8787,1.8969,48.1913,1.6763,49.7200,49.5986,1847.0000
3040092,2017-08-05,1,AUTOMOTIVE,2981286.0000,5.0000,0.0000,2017,8,5,-0.0000,1.0000,0.6602,0.7511,-0.5575,-0.8302,0,1,0,0.0000,NaN,4.0000,10.0000,5.0000,5.0000,5.0000,2.6652,5.2667,3.1275,4.0000,4.7143,Quito,Pichincha,D,13,Local,1,0,0,49.5700,49.5700,49.7200,45.7800,45.5200,41.8300,47.0273,1.9280,48.3807,1.6598,49.6000,49.5771,1251.0000
3041874,2017-08-06,1,AUTOMOTIVE,2983068.0000,6.0000,0.0000,2017,8,6,0.7818,0.6235,0.7998,0.6002,-0.5717,-0.8205,0,1,0,0.0000,NaN,1.0000,8.0000,4.0000,5.0000,5.0000,2.6652,4.9333,2.8402,5.0000,4.8571,Quito,Pichincha,D,13,NONE,0,0,0,49.5700,49.5700,49.7200,45.7800,44.2500,41.8300,47.1623,1.9604,48.6333,1.5180,49.5700,49.5557,507.0000
3043656,2017-08-07,1,AUTOMOTIVE,2984850.0000,7.0000,0.0000,2017,8,0,0.9749,-0.2225,0.9057,0.4239,-0.5857,-0.8105,0,0,0,0.0000,NaN,NaN,0.0000,6.0000,1.0000,5.0667,2.6644,4.8000,2.7308,NaN,NaN,Quito,Pichincha,D,13,NONE,0,0,0,49.3700,49.5700,50.2100,45.7800,44.2500,41.8300,47.3397,1.9282,48.8860,1.3104,49.5700,49.5343,1665.0000
3045438,2017-08-08,1,AUTOMOTIVE,2986632.0000,4.0000,0.0000,2017,8,1,0.4339,-0.9010,0.9733,0.2297,-0.5996,-0.8003,0,0,0,0.0000,NaN,NaN,4.0000,2.0000,4.0000,5.1000,2.6826,5.2667,2.4339,NaN,NaN,Quito,Pichincha,D,13,NONE,0,0,0,49.0700,49.3700,49.1900,46.2100,44.2500,43.0600,47.5103,1.8710,49.1253,0.9917,49.5700,49.4143,1766.0000
3047220,2017-08-09,1,AUTOMOTIVE,2988414.0000,7.0000,0.0000,2017,8,2,-0.4339,-0.9010,0.9997,0.0257,-0.6132,-0.7899,0,0,0,0.0000,NaN,NaN,10.0000,3.0000,6.0000,5.1667,2.6272,5.2667,2.4339,NaN,NaN,Quito,Pichincha,D,13,NONE,0,0,0,49.5900,49.0700,49.6000,47.7700,44.4000,42.7800,47.6710,1.7864,49.3160,0.5811,49.5700,49.3971,1766.0000
3049002,2017-08-10,1,AUTOMOTIVE,2990196.0000,9.0000,0.0000,2017,8,3,-0.9749,-0.2225,0.9838,-0.1793,-0.6267,-0.7792,0,0,0,0.0000,NaN,NaN,2.0000,7.0000,4.0000,5.3000,2.6149,5.0667,2.1202,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1,48.5400,49.5900,49.0300,48.5800,45.0600,41.7500,47.8440,1.7084,49.4373,0.3956,49.5700,49.3957,1764.0000

In [45]:
data.dtypes

,0
date,datetime64[ns]
store_nbr,int64
family,object
id,float64
sales,float64
onpromotion,float64
year,int64
month,int64
day_of_week,int64
sin_7,float64


In [46]:
data['date'] = pd.to_datetime(data['date'])
useless_features = ['date', 'sales', 'id', 'store_nbr']
all_features = list(data.drop(columns=useless_features).columns)
cat_features = ['year', 'month', 'day_of_week',  'family', 'store_city', 'store_state', 'store_type', 'store_cluster', 'holiday_locale']
num_features = [col for col in all_features if col not in cat_features]
data[cat_features] = data[cat_features].astype('category')

In [47]:
data.dtypes

,0
date,datetime64[ns]
store_nbr,int64
family,category
id,float64
sales,float64
onpromotion,float64
year,category
month,category
day_of_week,category
sin_7,float64


In [48]:
train = data[data['date'] <= end_of_train]
valid = data[data['date'] > end_of_train]
valid[(valid['family'] == 'AUTOMOTIVE') & (valid['store_nbr'] == 1)]

,date,store_nbr,family,id,sales,onpromotion,year,month,day_of_week,sin_7,cos_7,sin_30.5,cos_30.5,sin_365.25,cos_365.25,is_payday,is_weekend,is_christmas,earthquake_effect,sales_lag_1,sales_lag_7,sales_lag_15,sales_lag_30,sales_lag_365,sales_rolling_mean_30,sales_rolling_std_30,sales_rolling_mean_15,sales_rolling_std_15,sales_rolling_median_7,sales_rolling_mean_7,store_city,store_state,store_type,store_cluster,holiday_locale,is_holiday,is_event,store_in_event_area,dcoilwtico,oil_lag_1,oil_lag_7,oil_lag_15,oil_lag_30,oil_lag_365,oil_rolling_mean_30,oil_rolling_std_30,oil_rolling_mean_15,oil_rolling_std_15,oil_rolling_median_7,oil_rolling_mean_7,transactions
3032964,2017-08-01,1,AUTOMOTIVE,2974158.0000,5.0000,0.0000,2017,8,1,0.4339,-0.9010,-0.1028,0.9947,-0.4991,-0.8666,0,0,0,0.0000,NaN,10.0000,2.0000,4.0000,3.0000,4.6667,2.8203,5.0000,3.1848,5.0000,5.2857,Quito,Pichincha,D,13,NONE,0,0,0,49.1900,50.2100,47.7700,46.0200,46.0200,40.0500,46.5533,1.7222,47.6380,1.6941,49.7200,49.2529,1795.0000
3034746,2017-08-02,1,AUTOMOTIVE,2975940.0000,4.0000,0.0000,2017,8,2,-0.4339,-0.9010,0.1028,0.9947,-0.5139,-0.8578,0,0,0,0.0000,NaN,2.0000,3.0000,0.0000,10.0000,4.7000,2.8180,5.2000,3.0752,5.0000,4.5714,Quito,Pichincha,D,13,NONE,0,0,0,49.6000,49.1900,48.5800,46.4000,46.0200,39.5000,46.6590,1.7844,47.8493,1.6754,49.7200,49.4557,1892.0000
3036528,2017-08-03,1,AUTOMOTIVE,2977722.0000,3.0000,0.0000,2017,8,3,-0.9749,-0.2225,0.3041,0.9526,-0.5286,-0.8489,0,0,0,0.0000,NaN,5.0000,7.0000,5.0000,0.0000,4.8333,2.6792,5.2667,3.0347,5.0000,4.8571,Quito,Pichincha,D,13,NONE,0,0,0,49.0300,49.6000,49.0500,47.1000,46.0200,40.8000,46.7783,1.8584,48.0627,1.6814,49.7200,49.6014,1726.0000
3038310,2017-08-04,1,AUTOMOTIVE,2979504.0000,8.0000,0.0000,2017,8,4,-0.7818,0.6235,0.4925,0.8703,-0.5431,-0.8397,0,0,0,0.0000,NaN,7.0000,4.0000,1.0000,3.0000,4.7667,2.6997,5.0000,3.0472,4.0000,4.5714,Quito,Pichincha,D,13,NONE,0,0,0,49.5700,49.0300,49.7200,46.7300,45.1100,41.9200,46.8787,1.8969,48.1913,1.6763,49.7200,49.5986,1847.0000
3040092,2017-08-05,1,AUTOMOTIVE,2981286.0000,5.0000,0.0000,2017,8,5,-0.0000,1.0000,0.6602,0.7511,-0.5575,-0.8302,0,1,0,0.0000,NaN,4.0000,10.0000,5.0000,5.0000,5.0000,2.6652,5.2667,3.1275,4.0000,4.7143,Quito,Pichincha,D,13,Local,1,0,0,49.5700,49.5700,49.7200,45.7800,45.5200,41.8300,47.0273,1.9280,48.3807,1.6598,49.6000,49.5771,1251.0000
3041874,2017-08-06,1,AUTOMOTIVE,2983068.0000,6.0000,0.0000,2017,8,6,0.7818,0.6235,0.7998,0.6002,-0.5717,-0.8205,0,1,0,0.0000,NaN,1.0000,8.0000,4.0000,5.0000,5.0000,2.6652,4.9333,2.8402,5.0000,4.8571,Quito,Pichincha,D,13,NONE,0,0,0,49.5700,49.5700,49.7200,45.7800,44.2500,41.8300,47.1623,1.9604,48.6333,1.5180,49.5700,49.5557,507.0000
3043656,2017-08-07,1,AUTOMOTIVE,2984850.0000,7.0000,0.0000,2017,8,0,0.9749,-0.2225,0.9057,0.4239,-0.5857,-0.8105,0,0,0,0.0000,NaN,NaN,0.0000,6.0000,1.0000,5.0667,2.6644,4.8000,2.7308,NaN,NaN,Quito,Pichincha,D,13,NONE,0,0,0,49.3700,49.5700,50.2100,45.7800,44.2500,41.8300,47.3397,1.9282,48.8860,1.3104,49.5700,49.5343,1665.0000
3045438,2017-08-08,1,AUTOMOTIVE,2986632.0000,4.0000,0.0000,2017,8,1,0.4339,-0.9010,0.9733,0.2297,-0.5996,-0.8003,0,0,0,0.0000,NaN,NaN,4.0000,2.0000,4.0000,5.1000,2.6826,5.2667,2.4339,NaN,NaN,Quito,Pichincha,D,13,NONE,0,0,0,49.0700,49.3700,49.1900,46.2100,44.2500,43.0600,47.5103,1.8710,49.1253,0.9917,49.5700,49.4143,1766.0000
3047220,2017-08-09,1,AUTOMOTIVE,2988414.0000,7.0000,0.0000,2017,8,2,-0.4339,-0.9010,0.9997,0.0257,-0.6132,-0.7899,0,0,0,0.0000,NaN,NaN,10.0000,3.0000,6.0000,5.1667,2.6272,5.2667,2.4339,NaN,NaN,Quito,Pichincha,D,13,NONE,0,0,0,49.5900,49.0700,49.6000,47.7700,44.4000,42.7800,47.6710,1.7864,49.3160,0.5811,49.5700,49.3971,1766.0000
3049002,2017-08-10,1,AUTOMOTIVE,2990196.0000,9.0000,0.0000,2017,8,3,-0.9749,-0.2225,0.9838,-0.1793,-0.6267,-0.7792,0,0,0,0.0000,NaN,NaN,2.0000,7.0000,4.0000,5.3000,2.6149,5.0667,2.1202,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1,48.5400,49.5900,49.0300,48.5800,45.0600,41.7500,47.8440,1.7084,49.4373,0.3956,49.5700,49.3957,1764.0000

In [49]:
discrete_sales_families = []
continuous_sales_families = []
for family, group in data.groupby('family'):
  if np.allclose(group['sales'], group['sales'].astype('int')):
    discrete_sales_families.append(family)
  else:
    continuous_sales_families.append(family)

print(discrete_sales_families)
print(continuous_sales_families)

/tmp/ipython-input-1793903984.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for family, group in data.groupby('family'):


['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS', 'CELEBRATION', 'CLEANING', 'DAIRY', 'EGGS', 'GROCERY II', 'HARDWARE', 'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES', 'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE', 'LIQUOR,WINE,BEER', 'MAGAZINES', 'PERSONAL CARE', 'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'SCHOOL AND OFFICE SUPPLIES']
['BREAD/BAKERY', 'DELI', 'FROZEN FOODS', 'GROCERY I', 'MEATS', 'POULTRY', 'PREPARED FOODS', 'PRODUCE', 'SEAFOOD']


# Approach 1 - try all data
Yes, first I'll try to just upload all the data.

In [50]:
train_1 = train.copy()
valid_1 = valid.copy()
# for rolling statistics and lag features
prevalid_1_ = data[
    (data['date'] >= (end_of_train - pd.Timedelta(days=30))) &
    (data['date'] <= end_of_train)
    ].copy()

In [51]:
prevalid_1_.head()

,date,store_nbr,family,id,sales,onpromotion,year,month,day_of_week,sin_7,cos_7,sin_30.5,cos_30.5,sin_365.25,cos_365.25,is_payday,is_weekend,is_christmas,earthquake_effect,sales_lag_1,sales_lag_7,sales_lag_15,sales_lag_30,sales_lag_365,sales_rolling_mean_30,sales_rolling_std_30,sales_rolling_mean_15,sales_rolling_std_15,sales_rolling_median_7,sales_rolling_mean_7,store_city,store_state,store_type,store_cluster,holiday_locale,is_holiday,is_event,store_in_event_area,dcoilwtico,oil_lag_1,oil_lag_7,oil_lag_15,oil_lag_30,oil_lag_365,oil_rolling_mean_30,oil_rolling_std_30,oil_rolling_mean_15,oil_rolling_std_15,oil_rolling_median_7,oil_rolling_mean_7,transactions
2975940,2017-07-01,1,AUTOMOTIVE,2918916.0000,7.0000,0.0000,2017,7,5,-0.0000,1.0000,-0.2046,0.9789,0.0108,-0.9999,0,1,0,0.0000,11.0000,12.0000,4.0000,6.0000,8.0000,4.6000,4.0565,4.2667,3.6541,4.0000,5.2857,Quito,Pichincha,D,13,NONE,0,0,0,46.0200,46.0200,42.8600,44.7300,48.3200,49.0200,45.2030,1.7425,43.8993,1.0808,44.2500,44.1214,1362.0000
2975941,2017-07-01,1,BABY CARE,2918917.0000,0.0000,0.0000,2017,7,5,-0.0000,1.0000,-0.2046,0.9789,0.0108,-0.9999,0,1,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,Quito,Pichincha,D,13,NONE,0,0,0,46.0200,46.0200,42.8600,44.7300,48.3200,49.0200,45.2030,1.7425,43.8993,1.0808,44.2500,44.1214,1362.0000
2975942,2017-07-01,1,BEAUTY,2918918.0000,7.0000,1.0000,2017,7,5,-0.0000,1.0000,-0.2046,0.9789,0.0108,-0.9999,0,1,0,0.0000,2.0000,8.0000,5.0000,1.0000,5.0000,3.4000,2.0611,4.0667,2.0166,4.0000,4.4286,Quito,Pichincha,D,13,NONE,0,0,0,46.0200,46.0200,42.8600,44.7300,48.3200,49.0200,45.2030,1.7425,43.8993,1.0808,44.2500,44.1214,1362.0000
2975943,2017-07-01,1,BEVERAGES,2918919.0000,2596.0000,27.0000,2017,7,5,-0.0000,1.0000,-0.2046,0.9789,0.0108,-0.9999,0,1,0,0.0000,2496.0000,2674.0000,2532.0000,2250.0000,2298.0000,2287.5667,528.7883,2230.6667,562.8255,2203.0000,2130.5714,Quito,Pichincha,D,13,NONE,0,0,0,46.0200,46.0200,42.8600,44.7300,48.3200,49.0200,45.2030,1.7425,43.8993,1.0808,44.2500,44.1214,1362.0000
2975944,2017-07-01,1,BOOKS,2918920.0000,0.0000,0.0000,2017,7,5,-0.0000,1.0000,-0.2046,0.9789,0.0108,-0.9999,0,1,0,0.0000,0.0000,1.0000,1.0000,3.0000,0.0000,0.5000,0.9002,0.1333,0.3519,0.0000,0.1429,Quito,Pichincha,D,13,NONE,0,0,0,46.0200,46.0200,42.8600,44.7300,48.3200,49.0200,45.2030,1.7425,43.8993,1.0808,44.2500,44.1214,1362.0000


In [60]:
train_1['is_discrete'] = train_1['family'].isin(discrete_sales_families).astype('int')
valid_1['is_discrete'] = valid_1['family'].isin(discrete_sales_families).astype('int')
prevalid_1_['is_discrete'] = prevalid_1_['family'].isin(discrete_sales_families).astype('int')

In [53]:
X_train_1_catboost = train_1[cat_features + num_features + ['is_discrete']]
y_train_1_catboost = np.log1p(train_1['sales'])

In [54]:
train_pool = Pool(X_train_1_catboost, y_train_1_catboost, cat_features=cat_features)
catboost_model = CatBoostRegressor(loss_function='RMSE', random_seed=42, verbose=100, iterations=1000, task_type='GPU')
catboost_model.fit(train_pool)

Learning rate set to 0.115461
0:	learn: 2.4071142	total: 544ms	remaining: 9m 3s
100:	learn: 0.4128952	total: 52.8s	remaining: 7m 50s
200:	learn: 0.3961674	total: 1m 44s	remaining: 6m 56s
300:	learn: 0.3863845	total: 2m 36s	remaining: 6m 3s
400:	learn: 0.3803550	total: 3m 27s	remaining: 5m 9s
500:	learn: 0.3756789	total: 4m 19s	remaining: 4m 18s
600:	learn: 0.3723069	total: 5m 12s	remaining: 3m 27s
700:	learn: 0.3698289	total: 6m 4s	remaining: 2m 35s
800:	learn: 0.3674088	total: 6m 56s	remaining: 1m 43s
900:	learn: 0.3655992	total: 7m 48s	remaining: 51.5s
999:	learn: 0.3639034	total: 8m 40s	remaining: 0us


In [55]:
feature_importance_series = pd.Series(catboost_model.feature_importances_, index=X_train_1_catboost.columns)
feature_importance_series

,0
year,0.1716
month,0.1471
day_of_week,0.7767
family,2.0062
store_city,0.0690
store_state,0.0785
store_type,0.1316
store_cluster,0.0638
holiday_locale,0.0242
onpromotion,0.8187


In [56]:
print('Top 10 important features')
feature_importance_series.nlargest(10)

Top 10 important features


,0
sales_rolling_mean_7,26.8028
sales_lag_7,24.2080
sales_rolling_median_7,15.3944
sales_lag_1,12.1112
is_christmas,4.4919
sales_lag_15,3.0816
family,2.0062
transactions,1.9943
sales_rolling_mean_15,1.1208
cos_365.25,0.9776


In [57]:
print('Top 10 non-important features')
feature_importance_series.nsmallest(10)

Top 10 non-important features


,0
is_payday,0.0103
earthquake_effect,0.0168
cos_7,0.0197
holiday_locale,0.0242
is_event,0.0280
is_holiday,0.0440
oil_lag_15,0.0449
store_cluster,0.0638
is_discrete,0.0662
oil_rolling_mean_30,0.0674


In [58]:
valid_1.columns

Index(['date', 'store_nbr', 'family', 'id', 'sales', 'onpromotion', 'year',
       'month', 'day_of_week', 'sin_7', 'cos_7', 'sin_30.5', 'cos_30.5',
       'sin_365.25', 'cos_365.25', 'is_payday', 'is_weekend', 'is_christmas',
       'earthquake_effect', 'sales_lag_1', 'sales_lag_7', 'sales_lag_15',
       'sales_lag_30', 'sales_lag_365', 'sales_rolling_mean_30',
       'sales_rolling_std_30', 'sales_rolling_mean_15', 'sales_rolling_std_15',
       'sales_rolling_median_7', 'sales_rolling_mean_7', 'store_city',
       'store_state', 'store_type', 'store_cluster', 'holiday_locale',
       'is_holiday', 'is_event', 'store_in_event_area', 'dcoilwtico',
       'oil_lag_1', 'oil_lag_7', 'oil_lag_15', 'oil_lag_30', 'oil_lag_365',
       'oil_rolling_mean_30', 'oil_rolling_std_30', 'oil_rolling_mean_15',
       'oil_rolling_std_15', 'oil_rolling_median_7', 'oil_rolling_mean_7',
       'transactions', 'is_discrete'],
      dtype='object')

In [61]:
# For each pair (store_nbr, family), we store a queue of sales (maximum of the last 30)
history = defaultdict(lambda: deque(maxlen=30))

# Filling in the history with the initial data
for _, row in prevalid_1_.iterrows():
    key = (row.store_nbr, row.family)
    history[key].append(row.sales)

common_loss = 0

for d in range(DAYS_TO_PREDICT):
    prediction_date = start_valid_date + pd.Timedelta(days=d)
    valid_today = valid_1[valid_1['date'] == prediction_date].copy()

    # we create features incrementally
    sales_lag_1 = []
    sales_lag_7 = []
    rolling_mean_30 = []
    rolling_std_30 = []
    rolling_mean_15 = []
    rolling_std_15 = []
    rolling_median_7 = []
    rolling_mean_7 = []

    for _, row in valid_today.iterrows():
        key = (row.store_nbr, row.family)
        hist = history[key]

        # Lags
        sales_lag_1.append(hist[-1] if len(hist) >= 1 else np.nan)
        sales_lag_7.append(hist[-7] if len(hist) >= 7 else np.nan)

        # Rolling
        arr = np.array(hist)
        rolling_mean_30.append(arr[-30:].mean() if len(arr) > 0 else np.nan)
        rolling_std_30.append(arr[-30:].std() if len(arr) > 0 else np.nan)
        rolling_mean_15.append(arr[-15:].mean() if len(arr) > 0 else np.nan)
        rolling_std_15.append(arr[-15:].std() if len(arr) > 0 else np.nan)
        rolling_median_7.append(np.median(arr[-7:]) if len(arr) > 0 else np.nan)
        rolling_mean_7.append(arr[-7:].mean() if len(arr) > 0 else np.nan)

    valid_today['sales_lag_1'] = sales_lag_1
    valid_today['sales_lag_7'] = sales_lag_7
    valid_today['sales_rolling_mean_30'] = rolling_mean_30
    valid_today['sales_rolling_std_30'] = rolling_std_30
    valid_today['sales_rolling_mean_15'] = rolling_mean_15
    valid_today['sales_rolling_std_15'] = rolling_std_15
    valid_today['sales_rolling_median_7'] = rolling_median_7
    valid_today['sales_rolling_mean_7'] = rolling_mean_7

    # inference
    x_valid = valid_today[cat_features + num_features + ['is_discrete']]
    y_valid = np.array(valid_today['sales'])

    y_pred_log = catboost_model.predict(x_valid)
    y_pred = np.maximum(0, np.expm1(y_pred_log))

    mask_discrete = x_valid['is_discrete'] == 1
    y_pred[mask_discrete] = np.round(y_pred[mask_discrete])

    # refresh history
    for (_, row), pred in zip(valid_today.iterrows(), y_pred):
        key = (row.store_nbr, row.family)
        history[key].append(pred)

    #loss
    loss = root_mean_squared_log_error(y_valid, y_pred)
    print(f'Loss on {prediction_date}: {loss}')
    common_loss += loss

common_loss /= DAYS_TO_PREDICT
print(f'Common loss: {common_loss}')

Loss on 2017-08-01 00:00:00: 0.4296370818301105
Loss on 2017-08-02 00:00:00: 0.39664550127445813
Loss on 2017-08-03 00:00:00: 0.3995413015792306
Loss on 2017-08-04 00:00:00: 0.42865924571938824
Loss on 2017-08-05 00:00:00: 0.3837154793427524
Loss on 2017-08-06 00:00:00: 0.40943480600788096
Loss on 2017-08-07 00:00:00: 0.4295762479517211
Loss on 2017-08-08 00:00:00: 0.4467697406883647
Loss on 2017-08-09 00:00:00: 0.410658268395596
Loss on 2017-08-10 00:00:00: 0.4220397990930117
Loss on 2017-08-11 00:00:00: 0.44449466012620065
Loss on 2017-08-12 00:00:00: 0.41718079584543577
Loss on 2017-08-13 00:00:00: 0.41391764783983365
Loss on 2017-08-14 00:00:00: 0.4690524846238531
Loss on 2017-08-15 00:00:00: 0.4596856888329694
Common loss: 0.4240672499433872


In [62]:
del train_1
del valid_1
del history
del prevalid_1_
del X_train_1_catboost
del y_train_1_catboost

# Approach 2 - The same thing, but let's try to train only on fresh stable data from 2016.

In [63]:
train_2 = train[train['date'] >= '2016-01-01'].copy()
valid_2 = valid.copy()
# for rolling statistics and lag features
prevalid_2 = data[
    (data['date'] >= (end_of_train - pd.Timedelta(days=30))) &
    (data['date'] <= end_of_train)
    ].copy()

In [64]:
train_2.head()

,date,store_nbr,family,id,sales,onpromotion,year,month,day_of_week,sin_7,cos_7,sin_30.5,cos_30.5,sin_365.25,cos_365.25,is_payday,is_weekend,is_christmas,earthquake_effect,sales_lag_1,sales_lag_7,sales_lag_15,sales_lag_30,sales_lag_365,sales_rolling_mean_30,sales_rolling_std_30,sales_rolling_mean_15,sales_rolling_std_15,sales_rolling_median_7,sales_rolling_mean_7,store_city,store_state,store_type,store_cluster,holiday_locale,is_holiday,is_event,store_in_event_area,dcoilwtico,oil_lag_1,oil_lag_7,oil_lag_15,oil_lag_30,oil_lag_365,oil_rolling_mean_30,oil_rolling_std_30,oil_rolling_mean_15,oil_rolling_std_15,oil_rolling_median_7,oil_rolling_mean_7,transactions
1978020,2016-01-01,1,AUTOMOTIVE,1945944.0000,0.0000,0.0000,2016,1,4,0.7818,0.6235,0.2046,0.9789,0.0172,0.9999,0,0,1,0.0000,5.0000,0.0000,3.0000,1.0000,0.0000,4.2333,3.1914,3.9333,2.8652,5.0000,4.7143,Quito,Pichincha,D,13,National,1,0,1,37.1300,37.1300,37.6200,34.9800,39.9300,53.4500,37.0390,1.7661,36.3340,1.2751,37.6200,37.2600,0.0000
1978021,2016-01-01,1,BABY CARE,1945945.0000,0.0000,0.0000,2016,1,4,0.7818,0.6235,0.2046,0.9789,0.0172,0.9999,0,0,1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,Quito,Pichincha,D,13,National,1,0,1,37.1300,37.1300,37.6200,34.9800,39.9300,53.4500,37.0390,1.7661,36.3340,1.2751,37.6200,37.2600,0.0000
1978022,2016-01-01,1,BEAUTY,1945946.0000,0.0000,0.0000,2016,1,4,0.7818,0.6235,0.2046,0.9789,0.0172,0.9999,0,0,1,0.0000,0.0000,0.0000,5.0000,4.0000,0.0000,2.6000,1.7538,2.8000,2.0424,0.0000,1.5714,Quito,Pichincha,D,13,National,1,0,1,37.1300,37.1300,37.6200,34.9800,39.9300,53.4500,37.0390,1.7661,36.3340,1.2751,37.6200,37.2600,0.0000
1978023,2016-01-01,1,BEVERAGES,1945947.0000,0.0000,0.0000,2016,1,4,0.7818,0.6235,0.2046,0.9789,0.0172,0.9999,0,0,1,0.0000,2179.0000,0.0000,1958.0000,2520.0000,0.0000,2025.7000,615.2982,1914.1333,752.8484,1842.0000,1562.4286,Quito,Pichincha,D,13,National,1,0,1,37.1300,37.1300,37.6200,34.9800,39.9300,53.4500,37.0390,1.7661,36.3340,1.2751,37.6200,37.2600,0.0000
1978024,2016-01-01,1,BOOKS,1945948.0000,0.0000,0.0000,2016,1,4,0.7818,0.6235,0.2046,0.9789,0.0172,0.9999,0,0,1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,Quito,Pichincha,D,13,National,1,0,1,37.1300,37.1300,37.6200,34.9800,39.9300,53.4500,37.0390,1.7661,36.3340,1.2751,37.6200,37.2600,0.0000


In [65]:
train_2.shape

(1054944, 51)

In [66]:
train_2['is_discrete'] = train_2['family'].isin(discrete_sales_families).astype('int')
valid_2['is_discrete'] = valid_2['family'].isin(discrete_sales_families).astype('int')
prevalid_2['is_discrete'] = prevalid_2['family'].isin(discrete_sales_families).astype('int')

In [67]:
X_train_2_catboost = train_2[cat_features + num_features + ['is_discrete']]
y_train_2_catboost = np.log1p(train_2['sales'])

In [68]:
train_pool = Pool(X_train_2_catboost, y_train_2_catboost, cat_features=cat_features)
catboost_model_2 = CatBoostRegressor(loss_function='RMSE', random_seed=42, verbose=100, iterations=1000, task_type='GPU')
catboost_model_2.fit(train_pool)

Learning rate set to 0.100544
0:	learn: 2.3439333	total: 169ms	remaining: 2m 48s
100:	learn: 0.4179238	total: 15.4s	remaining: 2m 17s
200:	learn: 0.4050282	total: 31.6s	remaining: 2m 5s
300:	learn: 0.3982596	total: 48.1s	remaining: 1m 51s
400:	learn: 0.3940286	total: 1m 5s	remaining: 1m 37s
500:	learn: 0.3902812	total: 1m 20s	remaining: 1m 20s
600:	learn: 0.3876228	total: 1m 34s	remaining: 1m 2s
700:	learn: 0.3854915	total: 1m 49s	remaining: 46.6s
800:	learn: 0.3839523	total: 2m 3s	remaining: 30.8s
900:	learn: 0.3825152	total: 2m 18s	remaining: 15.2s
999:	learn: 0.3811594	total: 2m 34s	remaining: 0us


In [69]:
feature_importance_series = pd.Series(catboost_model_2.feature_importances_, index=X_train_2_catboost.columns)
feature_importance_series

,0
year,0.0163
month,0.0226
day_of_week,0.5558
family,0.7505
store_city,0.0975
store_state,0.1280
store_type,0.1716
store_cluster,0.1446
holiday_locale,0.0371
onpromotion,1.5142


In [70]:
print('Top 10 important features')
feature_importance_series.nlargest(10)

Top 10 important features


,0
sales_lag_7,30.2018
sales_rolling_mean_7,17.1769
sales_rolling_median_7,14.0281
sales_rolling_mean_30,9.5336
sales_rolling_mean_15,8.0825
sales_lag_1,6.0298
is_christmas,5.1991
transactions,1.6124
onpromotion,1.5142
is_weekend,0.9045


In [71]:
print('Top 10 non-important features')
feature_importance_series.nsmallest(10)

Top 10 non-important features


,0
is_event,0.0025
is_payday,0.0112
year,0.0163
month,0.0226
oil_lag_1,0.0352
holiday_locale,0.0371
is_holiday,0.0416
earthquake_effect,0.0435
oil_rolling_mean_7,0.0490
oil_rolling_median_7,0.0524


In [72]:
# For each pair (store_nbr, family), we store a queue of sales (maximum of the last 30)
history = defaultdict(lambda: deque(maxlen=30))

# Filling in the history with the initial data
for _, row in prevalid_2.iterrows():
    key = (row.store_nbr, row.family)
    history[key].append(row.sales)

common_loss = 0

for d in range(DAYS_TO_PREDICT):
    prediction_date = start_valid_date + pd.Timedelta(days=d)
    valid_today = valid_2[valid_2['date'] == prediction_date].copy()

    # we create features incrementally
    sales_lag_1 = []
    sales_lag_7 = []
    rolling_mean_30 = []
    rolling_std_30 = []
    rolling_mean_15 = []
    rolling_std_15 = []
    rolling_median_7 = []
    rolling_mean_7 = []

    for _, row in valid_today.iterrows():
        key = (row.store_nbr, row.family)
        hist = history[key]

        # Lags
        sales_lag_1.append(hist[-1] if len(hist) >= 1 else np.nan)
        sales_lag_7.append(hist[-7] if len(hist) >= 7 else np.nan)

        # Rolling
        arr = np.array(hist)
        rolling_mean_30.append(arr[-30:].mean() if len(arr) > 0 else np.nan)
        rolling_std_30.append(arr[-30:].std() if len(arr) > 0 else np.nan)
        rolling_mean_15.append(arr[-15:].mean() if len(arr) > 0 else np.nan)
        rolling_std_15.append(arr[-15:].std() if len(arr) > 0 else np.nan)
        rolling_median_7.append(np.median(arr[-7:]) if len(arr) > 0 else np.nan)
        rolling_mean_7.append(arr[-7:].mean() if len(arr) > 0 else np.nan)

    valid_today['sales_lag_1'] = sales_lag_1
    valid_today['sales_lag_7'] = sales_lag_7
    valid_today['sales_rolling_mean_30'] = rolling_mean_30
    valid_today['sales_rolling_std_30'] = rolling_std_30
    valid_today['sales_rolling_mean_15'] = rolling_mean_15
    valid_today['sales_rolling_std_15'] = rolling_std_15
    valid_today['sales_rolling_median_7'] = rolling_median_7
    valid_today['sales_rolling_mean_7'] = rolling_mean_7

    # inference
    x_valid = valid_today[cat_features + num_features + ['is_discrete']]
    y_valid = np.array(valid_today['sales'])

    y_pred_log = catboost_model_2.predict(x_valid)
    y_pred = np.maximum(0, np.expm1(y_pred_log))

    mask_discrete = x_valid['is_discrete'] == 1
    y_pred[mask_discrete] = np.round(y_pred[mask_discrete])

    # refresh history
    for (_, row), pred in zip(valid_today.iterrows(), y_pred):
        key = (row.store_nbr, row.family)
        history[key].append(pred)

    #loss
    loss = root_mean_squared_log_error(y_valid, y_pred)
    print(f'Loss on {prediction_date}: {loss}')
    common_loss += loss

common_loss /= DAYS_TO_PREDICT
print(f'Common loss: {common_loss}')

Loss on 2017-08-01 00:00:00: 0.38371374442691897
Loss on 2017-08-02 00:00:00: 0.3881194689419974
Loss on 2017-08-03 00:00:00: 0.40061492633997414
Loss on 2017-08-04 00:00:00: 0.4229008817482046
Loss on 2017-08-05 00:00:00: 0.38933076459255345
Loss on 2017-08-06 00:00:00: 0.40518150296637123
Loss on 2017-08-07 00:00:00: 0.4040317920509503
Loss on 2017-08-08 00:00:00: 0.4287231238086362
Loss on 2017-08-09 00:00:00: 0.3994003428436572
Loss on 2017-08-10 00:00:00: 0.41064304350961484
Loss on 2017-08-11 00:00:00: 0.42855779353125373
Loss on 2017-08-12 00:00:00: 0.4233175130353235
Loss on 2017-08-13 00:00:00: 0.4262771391642664
Loss on 2017-08-14 00:00:00: 0.43988749694987245
Loss on 2017-08-15 00:00:00: 0.4273611646392805
Common loss: 0.4118707132365917


Yes, there is enough stable data for the last year, the model is even better in the validation sample. But still, of course, the target is very scattered, let's try to divide it into 2 datasets.

In [73]:
del train_2
del valid_2
del history
del prevalid_2
del X_train_2_catboost
del y_train_2_catboost

# Approach 3 - divide the data by the type of target

I thought it would be better to separate the approaches with a function so that the memory is automatically cleared.

In [74]:
def approach_3_catboost():
  _train = train.copy()
  _valid = valid.copy()
  prevalid = data[
    (data['date'] >= (end_of_train - pd.Timedelta(days=30))) &
    (data['date'] <= end_of_train)
    ].copy()

  # discrete_train = _train[(_train['family'].isin(discrete_sales_families))]
  discrete_train = _train[(_train['family'].isin(discrete_sales_families)) & (_train['date'] >= '2016-01-01')]
  continuous_train = _train[_train['family'].isin(continuous_sales_families)]

  print('discrete_train', discrete_train.shape)
  print('continuous train', continuous_train.shape)
  print('valid', _valid.shape)

  continuous_X_train = continuous_train[cat_features + num_features]
  discrete_X_train = discrete_train[cat_features + num_features]

  discrete_y_train = np.log1p(discrete_train['sales'])
  continuous_y_train = np.log1p(continuous_train['sales'])

  discrete_train_pool = Pool(discrete_X_train, discrete_y_train, cat_features=cat_features)
  continuous_train_pool = Pool(continuous_X_train, continuous_y_train, cat_features=cat_features)



  discrete_model = CatBoostRegressor(loss_function='RMSE', random_seed=42, verbose=100, iterations=1000, task_type='GPU')
  continuous_model = CatBoostRegressor(loss_function='RMSE', random_seed=42, verbose=100, iterations=1000, task_type='GPU')



  print('\ndiscrete model\n')
  discrete_model.fit(discrete_train_pool)

  feature_importance_series = pd.Series(discrete_model.feature_importances_, index=discrete_X_train.columns)
  print('\ndiscrete model feature importances:')
  print(feature_importance_series)
  print('\nTop 10 important features')
  print(feature_importance_series.nlargest(10))
  print('\nTop 10 non-important features')
  print(feature_importance_series.nsmallest(10))




  print('\ncontinuous model\n')
  continuous_model.fit(continuous_train_pool)

  feature_importance_series = pd.Series(continuous_model.feature_importances_, index=continuous_X_train.columns)
  print('\ncontinuous model feature importances:')
  print(feature_importance_series)
  print('\nTop 10 important features')
  print(feature_importance_series.nlargest(10))
  print('\nTop 10 non-important features')
  print(feature_importance_series.nsmallest(10))



  # VALIDATION
  history = defaultdict(lambda: deque(maxlen=30))

  # Filling in the history with the initial data
  for _, row in prevalid.iterrows():
      key = (row.store_nbr, row.family)
      history[key].append(row.sales)


  common_loss = 0
  discrete_common_loss = 0
  continuous_common_loss = 0

  for d in range(DAYS_TO_PREDICT):
      prediction_date = start_valid_date + pd.Timedelta(days=d)
      valid_today = _valid[_valid['date'] == prediction_date].copy()

      # we create features incrementally
      sales_lag_1 = []
      sales_lag_7 = []
      rolling_mean_30 = []
      rolling_std_30 = []
      rolling_mean_15 = []
      rolling_std_15 = []
      rolling_median_7 = []
      rolling_mean_7 = []

      for _, row in valid_today.iterrows():
          key = (row.store_nbr, row.family)
          hist = history[key]

          # Lags
          sales_lag_1.append(hist[-1] if len(hist) >= 1 else np.nan)
          sales_lag_7.append(hist[-7] if len(hist) >= 7 else np.nan)

          # Rolling
          arr = np.array(hist)
          rolling_mean_30.append(arr[-30:].mean() if len(arr) > 0 else np.nan)
          rolling_std_30.append(arr[-30:].std() if len(arr) > 0 else np.nan)
          rolling_mean_15.append(arr[-15:].mean() if len(arr) > 0 else np.nan)
          rolling_std_15.append(arr[-15:].std() if len(arr) > 0 else np.nan)
          rolling_median_7.append(np.median(arr[-7:]) if len(arr) > 0 else np.nan)
          rolling_mean_7.append(arr[-7:].mean() if len(arr) > 0 else np.nan)

      valid_today['sales_lag_1'] = sales_lag_1
      valid_today['sales_lag_7'] = sales_lag_7
      valid_today['sales_rolling_mean_30'] = rolling_mean_30
      valid_today['sales_rolling_std_30'] = rolling_std_30
      valid_today['sales_rolling_mean_15'] = rolling_mean_15
      valid_today['sales_rolling_std_15'] = rolling_std_15
      valid_today['sales_rolling_median_7'] = rolling_median_7
      valid_today['sales_rolling_mean_7'] = rolling_mean_7

      # inference

      x_valid = valid_today[cat_features + num_features]
      y_valid = np.array(valid_today['sales'])

      discrete_x_valid = x_valid[(valid_today['family'].isin(discrete_sales_families))]
      continuous_x_valid = x_valid[(valid_today['family'].isin(continuous_sales_families))]
      discrete_y_valid = y_valid[(valid_today['family'].isin(discrete_sales_families))]
      continuous_y_valid = y_valid[(valid_today['family'].isin(continuous_sales_families))]

      discrete_y_pred = np.round(np.maximum(0, np.expm1(discrete_model.predict(discrete_x_valid)) ))
      continuous_y_pred = np.maximum(0, np.expm1(continuous_model.predict(continuous_x_valid)))

      y_pred = pd.concat([
        pd.Series(discrete_y_pred, index=discrete_x_valid.index),
        pd.Series(continuous_y_pred, index=continuous_x_valid.index)
      ]).sort_index()


      # refresh history
      for (_, row), pred in zip(valid_today.iterrows(), y_pred):
          key = (row.store_nbr, row.family)
          history[key].append(pred)

      #loss
      discrete_loss = root_mean_squared_log_error(discrete_y_valid, discrete_y_pred)
      continuous_loss = root_mean_squared_log_error(continuous_y_valid, continuous_y_pred)
      loss = root_mean_squared_log_error(y_valid, y_pred)
      print('\n')
      print(f'Discrete model loss on {prediction_date}: {discrete_loss}')
      print(f'Continuous model loss on {prediction_date}: {continuous_loss}')
      print(f'Loss on {prediction_date}: {loss}')

      discrete_common_loss += discrete_loss
      continuous_common_loss += continuous_loss
      common_loss += loss

  common_loss /= DAYS_TO_PREDICT
  discrete_common_loss /= DAYS_TO_PREDICT
  continuous_common_loss /= DAYS_TO_PREDICT
  print('\n\n')
  print(f'Common loss: {common_loss}')
  print(f'Common dicsrete loss: {discrete_common_loss}')
  print(f'Common continuous loss: {continuous_common_loss}')
  print('\n')

In [75]:
approach_3_catboost()

discrete_train (767232, 51)
continuous train (827172, 51)
valid (26730, 51)

discrete model

Learning rate set to 0.096436
0:	learn: 2.1972965	total: 114ms	remaining: 1m 53s
100:	learn: 0.4525482	total: 8.17s	remaining: 1m 12s
200:	learn: 0.4387510	total: 17.2s	remaining: 1m 8s
300:	learn: 0.4323281	total: 26.5s	remaining: 1m 1s
400:	learn: 0.4279581	total: 34.4s	remaining: 51.4s
500:	learn: 0.4250109	total: 43.8s	remaining: 43.6s
600:	learn: 0.4228835	total: 53.1s	remaining: 35.2s
700:	learn: 0.4210749	total: 1m 1s	remaining: 26s
800:	learn: 0.4195992	total: 1m 10s	remaining: 17.5s
900:	learn: 0.4181016	total: 1m 19s	remaining: 8.75s
999:	learn: 0.4169936	total: 1m 27s	remaining: 0us

discrete model feature importances:
year                      0.0190
month                     0.0250
day_of_week               0.6913
family                    1.1490
store_city                0.1166
store_state               0.1481
store_type                0.2264
store_cluster             0.1249
holid

# Approach 4 - try 33 models

In [76]:
valid.isnull().sum()

,0
date,0
store_nbr,0
family,0
id,0
sales,0
onpromotion,0
year,0
month,0
day_of_week,0
sin_7,0


In [77]:
def approach_4_catboost():
    _train = train.copy()
    _valid = valid.copy()

    prevalid = data[
        (data['date'] >= (end_of_train - pd.Timedelta(days=30))) &
        (data['date'] <= end_of_train)
    ].copy()

    families = _train['family'].unique()
    models = {}  # family -> CatBoost model

    _cat_features = cat_features.copy()
    _cat_features.remove('family')

    for fam in families:
        fam_train = _train[_train['family'] == fam]
        X_train = fam_train[_cat_features + num_features]
        y_train = np.log1p(fam_train['sales'])

        train_pool = Pool(X_train, y_train, cat_features=_cat_features)

        print(f"\nTraining model for family: {fam}\n")
        print('Data size:', train_pool.shape, '\n')
        model = CatBoostRegressor(
            loss_function='RMSE',
            random_seed=42,
            verbose=500,
            iterations=1000,
            task_type='GPU'
        )
        model.fit(train_pool)
        models[fam] = model

    history = defaultdict(lambda: deque(maxlen=30))
    for _, row in prevalid.iterrows():
        key = (row.store_nbr, row.family)
        history[key].append(row.sales)

    common_loss = 0
    for d in range(DAYS_TO_PREDICT):
        prediction_date = start_valid_date + pd.Timedelta(days=d)
        valid_today = _valid[_valid['date'] == prediction_date].copy()

        sales_lag_1, sales_lag_7 = [], []
        rolling_mean_30, rolling_std_30 = [], []
        rolling_mean_15, rolling_std_15 = [], []
        rolling_median_7, rolling_mean_7 = [], []

        for _, row in valid_today.iterrows():
            key = (row.store_nbr, row.family)
            hist = history[key]

            sales_lag_1.append(hist[-1] if len(hist) >= 1 else np.nan)
            sales_lag_7.append(hist[-7] if len(hist) >= 7 else np.nan)

            arr = np.array(hist)
            rolling_mean_30.append(arr[-30:].mean() if len(arr) > 0 else np.nan)
            rolling_std_30.append(arr[-30:].std() if len(arr) > 0 else np.nan)
            rolling_mean_15.append(arr[-15:].mean() if len(arr) > 0 else np.nan)
            rolling_std_15.append(arr[-15:].std() if len(arr) > 0 else np.nan)
            rolling_median_7.append(np.median(arr[-7:]) if len(arr) > 0 else np.nan)
            rolling_mean_7.append(arr[-7:].mean() if len(arr) > 0 else np.nan)

        valid_today['sales_lag_1'] = sales_lag_1
        valid_today['sales_lag_7'] = sales_lag_7
        valid_today['sales_rolling_mean_30'] = rolling_mean_30
        valid_today['sales_rolling_std_30'] = rolling_std_30
        valid_today['sales_rolling_mean_15'] = rolling_mean_15
        valid_today['sales_rolling_std_15'] = rolling_std_15
        valid_today['sales_rolling_median_7'] = rolling_median_7
        valid_today['sales_rolling_mean_7'] = rolling_mean_7

        y_valid = np.array(valid_today['sales'])
        y_pred = []

        for idx, row in valid_today.iterrows():
            fam = row['family']
            model = models[fam]
            x_row = row[_cat_features + num_features].to_frame().T
            pred = np.expm1(model.predict(x_row))[0]

            if fam in discrete_sales_families:
              pred = round(pred)

            pred = max(0, pred)
            y_pred.append(pred)

            key = (row.store_nbr, row.family)
            history[key].append(pred)

        y_pred = np.array(y_pred)
        loss = root_mean_squared_log_error(y_valid, y_pred)
        print(f"Loss on {prediction_date}: {loss}")
        common_loss += loss

    common_loss /= DAYS_TO_PREDICT
    print(f"Common loss: {common_loss}")

    return models

In [78]:
catboost_models_4  = approach_4_catboost()


Training model for family: AUTOMOTIVE

Data size: (91908, 46) 

Learning rate set to 0.073032
0:	learn: 0.8468333	total: 25.1ms	remaining: 25s
500:	learn: 0.4981984	total: 11.1s	remaining: 11.1s
999:	learn: 0.4861685	total: 21.4s	remaining: 0us

Training model for family: BABY CARE

Data size: (91908, 46) 

Learning rate set to 0.073032
0:	learn: 0.2395136	total: 17.1ms	remaining: 17.1s
500:	learn: 0.1728735	total: 8.42s	remaining: 8.38s
999:	learn: 0.1599786	total: 19.4s	remaining: 0us

Training model for family: BEAUTY

Data size: (91908, 46) 

Learning rate set to 0.073032
0:	learn: 0.8520507	total: 17ms	remaining: 17s
500:	learn: 0.4403171	total: 10.9s	remaining: 10.8s
999:	learn: 0.4295221	total: 21.4s	remaining: 0us

Training model for family: BEVERAGES

Data size: (91908, 46) 

Learning rate set to 0.073032
0:	learn: 2.0662860	total: 17.5ms	remaining: 17.5s
500:	learn: 0.1590787	total: 10.5s	remaining: 10.4s
999:	learn: 0.1448950	total: 19s	remaining: 0us

Training model for fa

# Approach 5 - try 33 models - individual for each family (info taken step 2 - Analysis)

In [79]:
def approach_4_catboost():
    _train = train.copy()
    _valid = valid.copy()

    prevalid = data[
        (data['date'] >= (end_of_train - pd.Timedelta(days=30))) &
        (data['date'] <= end_of_train)
    ].copy()

    families = _train['family'].unique()
    models = {}  # family -> CatBoost model

    _cat_features = cat_features.copy()
    _cat_features.remove('family')

    for fam in families:
        fam_train = _train[_train['family'] == fam]
        if fam in ['BABY CARE', 'MAGAZINES']:
          fam_train = fam_train[fam_train['date'] >= '2016-01-01']

        elif fam in ['BOOKS', 'LAWN AND GARDEN']:
          fam_train = fam_train[fam_train['date'] >= '2016-10-08']

        elif fam in ['CELEBRATION', 'HOME CARE', 'LADIESWEAR', 'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES']:
          fam_train = fam_train[fam_train['date'] >= '2015-06-01']

        elif fam in ['HOME AND KITCHEN I', 'HOME AND KITCHEN II']:
          fam_train = fam_train[fam_train['date'] >= '2015-01-01']


        X_train = fam_train[_cat_features + num_features]
        y_train = np.log1p(fam_train['sales'])

        train_pool = Pool(X_train, y_train, cat_features=_cat_features)

        print(f"\nTraining model for family: {fam}\n")
        print('Data size:', train_pool.shape, '\n')
        model = CatBoostRegressor(
            loss_function='RMSE',
            random_seed=42,
            verbose=500,
            iterations=1000,
            task_type='GPU'
        )
        model.fit(train_pool)
        models[fam] = model

    history = defaultdict(lambda: deque(maxlen=30))
    for _, row in prevalid.iterrows():
        key = (row.store_nbr, row.family)
        history[key].append(row.sales)

    common_loss = 0
    for d in range(DAYS_TO_PREDICT):
        prediction_date = start_valid_date + pd.Timedelta(days=d)
        valid_today = _valid[_valid['date'] == prediction_date].copy()

        sales_lag_1, sales_lag_7 = [], []
        rolling_mean_30, rolling_std_30 = [], []
        rolling_mean_15, rolling_std_15 = [], []
        rolling_median_7, rolling_mean_7 = [], []

        for _, row in valid_today.iterrows():
            key = (row.store_nbr, row.family)
            hist = history[key]

            sales_lag_1.append(hist[-1] if len(hist) >= 1 else np.nan)
            sales_lag_7.append(hist[-7] if len(hist) >= 7 else np.nan)

            arr = np.array(hist)
            rolling_mean_30.append(arr[-30:].mean() if len(arr) > 0 else np.nan)
            rolling_std_30.append(arr[-30:].std() if len(arr) > 0 else np.nan)
            rolling_mean_15.append(arr[-15:].mean() if len(arr) > 0 else np.nan)
            rolling_std_15.append(arr[-15:].std() if len(arr) > 0 else np.nan)
            rolling_median_7.append(np.median(arr[-7:]) if len(arr) > 0 else np.nan)
            rolling_mean_7.append(arr[-7:].mean() if len(arr) > 0 else np.nan)

        valid_today['sales_lag_1'] = sales_lag_1
        valid_today['sales_lag_7'] = sales_lag_7
        valid_today['sales_rolling_mean_30'] = rolling_mean_30
        valid_today['sales_rolling_std_30'] = rolling_std_30
        valid_today['sales_rolling_mean_15'] = rolling_mean_15
        valid_today['sales_rolling_std_15'] = rolling_std_15
        valid_today['sales_rolling_median_7'] = rolling_median_7
        valid_today['sales_rolling_mean_7'] = rolling_mean_7

        y_valid = np.array(valid_today['sales'])
        y_pred = []

        for idx, row in valid_today.iterrows():
            fam = row['family']
            model = models[fam]
            x_row = row[_cat_features + num_features].to_frame().T
            pred = np.expm1(model.predict(x_row))[0]

            if fam in discrete_sales_families:
              pred = round(pred)

            pred = max(0, pred)
            y_pred.append(pred)

            key = (row.store_nbr, row.family)
            history[key].append(pred)

        y_pred = np.array(y_pred)
        loss = root_mean_squared_log_error(y_valid, y_pred)
        print(f"Loss on {prediction_date}: {loss}")
        common_loss += loss

    common_loss /= DAYS_TO_PREDICT
    print(f"Common loss: {common_loss}")

    return models

In [80]:
catboost_models_4  = approach_4_catboost()


Training model for family: AUTOMOTIVE

Data size: (91908, 46) 

Learning rate set to 0.073032
0:	learn: 0.8468333	total: 24.5ms	remaining: 24.5s
500:	learn: 0.4981984	total: 10.2s	remaining: 10.2s
999:	learn: 0.4861685	total: 20s	remaining: 0us

Training model for family: BABY CARE

Data size: (31968, 46) 

Learning rate set to 0.063596
0:	learn: 0.3484869	total: 27.1ms	remaining: 27s
500:	learn: 0.2770358	total: 17.8s	remaining: 17.7s
999:	learn: 0.2701070	total: 33.7s	remaining: 0us

Training model for family: BEAUTY

Data size: (91908, 46) 

Learning rate set to 0.073032
0:	learn: 0.8520507	total: 16.9ms	remaining: 16.9s
500:	learn: 0.4403170	total: 10.5s	remaining: 10.5s
999:	learn: 0.4295234	total: 19.2s	remaining: 0us

Training model for family: BEVERAGES

Data size: (91908, 46) 

Learning rate set to 0.073032
0:	learn: 2.0662860	total: 88.2ms	remaining: 1m 28s
500:	learn: 0.1590787	total: 9.75s	remaining: 9.71s
999:	learn: 0.1448950	total: 20.7s	remaining: 0us

Training model f

# In general, the 33-model approach is the best.